# [02] 데이터셋 구성하기


어떠한 Dataset을 학습하기 위해서는 일반적으로 다음과 같은 과정을 거쳐 Data Preprocessing을 진행합니다.
- Dataset Class에서 index에 따라 data와 label 정보를 읽어올 수 있도록 하기
- 학습의 성능 향상을 위해 Dataset Class에 Augmentation을 추가하기
- Dataset Class를 Batch 단위로 읽어오는 DataLoader 만들기

이번 실습에서는 모델에 바로 입력할 수 있도록 Batch 단위로 데이터를 읽어오고, 결과값을 Label과 비교해 Loss를 계산할 수 있도록 DataSet과 DataLoader를 만드는 작업을 배워보도록 하겠습니다.

In [1]:
from IPython.display import HTML, display

# Image from JournalDev https://www.journaldev.com/36576/pytorch-dataloader
display(HTML("<img src='img/[02]-dataloader.png'>"))

---------------
Pytorch의 Dataset class를 구성하기 위해서는 2가지 함수를 작성해야 합니다.
- `__init__()`: 이 함수를 통해 데이터와 라벨 정보를 attribute로 얻습니다.
- `__getitem__(idx)`: 이 함수를 통해 해당 idx에 해당하는 데이터와 라벨을 얻습니다.

특히, 이후 DataLoader에서 `__getitem__`함수를 이용해 데이터를 샘플링하여 Batch를 구성하게 됩니다.

-------------------
## [Task 1] PascalVOCDataset class 정의하기
아래는 PascalVOCDataset의 `__init__`함수를 완성한 상태의 코드입니다.

### ToDo: 모델의 출력값 확인하기

이미지데이터와 라벨 정보를 index를 이용해 가져오도록 `__getitem__` 함수를 완성합니다. 라벨 정보는 다음의 type을 가지도록 합니다.
- boxes : `torch.FloatTensor`
- labels : `torch.LongTensor`
- difficulties : `torch.BoolTensor`


In [4]:
import torch
from torch.utils.data import Dataset
import json, os
from PIL import Image


class PascalVOCDataset(Dataset):
    """
    PasCalVOC 데이터로더 구성을 위한 Dataset class입니다.
    """

    def __init__(self, data_folder, split):
        """
        :data_folder : 데이터가 저장된 디렉토리
        :split: 'TRAIN' 또는 'TEST' 중 사용할 데이터 종류
        """
        # 입력한 split 값을 대문자로 바꿉니다.
        self.split = split.upper()
        
        # split 입력값에 대한 확인
        assert self.split in {'TRAIN', 'TEST'}
        
        self.data_folder = data_folder
        
        # json파일을 불러옵니다. images : 이미지 경로, objects : 라벨 정보
        with open(os.path.join(data_folder, self.split + '_images.json'), 'r') as j:
            self.images = json.load(j)
        with open(os.path.join(data_folder, self.split + '_objects.json'), 'r') as j:
            self.objects = json.load(j)

        assert len(self.images) == len(self.objects)

    def __getitem__(self, i):
        # [ToDo]: index i에 해당하는 이미지를 읽기 모드로 불러옵니다.
        image = Image.open(?????????, mode='r')
        
        # 이미지를 'RGB' 형태로 바꿉니다.
        image = image.convert('RGB')

        # (bounding boxes, labels, difficulties)의 라벨 정보를 얻습니다.
        objects = self.objects[i]
        
        # [ToDo]: 'boxes'에 저장된 box 좌표를 FloatTensor로 받습니다.
        boxes = torch.????????(???????????)  # (n_objects, 4)
        
        # [ToDo]: 'labels'에 저장된 라벨값을 LongTensor로 받습니다.
        labels = torch.????????(??????????)  # (n_objects)
        
        # [ToDo]: 'difficulties'에 저장된 difficulty 값을 BoolTensor로 받습니다.
        difficulties = torch.?????????(??????????)  # (n_objects)
        
        # (practice_transform 함수를 완성하고 아래 주석을 해제하세요!)
        #image, boxes = practice_transform(image, boxes)
        
        return image, boxes, labels, difficulties

    def __len__(self):
        return len(self.images)

`PascalVOCDataset` class를 완성한 다음 아래 주석들을 해제해 출력값의 형태를 확인해 보세요.

In [11]:
train_dataset = PascalVOCDataset(data_folder='./data/VOC', split='TRAIN')
test_dataset = PascalVOCDataset(data_folder='./data/VOC', split='TEST')

images, boxes, labels, difficulties = train_dataset.__getitem__(100)

#print(images)
print(boxes)
#print(labels)

tensor([[0.6660, 0.4955, 0.8960, 0.9940]])


-------------------
## [Task 2] Object Detection을 위한 Augmentation 

이번에는 Augmentation을 추가해 보도록 하겠습니다. 객체 인식(Object Detection)의 Augmentation은 다른 Augmenation의 방법들과는 달리 Augmentation 방법에 따라서 Label(Bounding Box의 좌표) 또한 함께 바뀌어야 한다는 특성이 있습니다. 이를 고려하여 간단한 몇 가지의 Augmentation이 어떻게 구성되는지 살펴봅시다.

### ToDo: Augmeatation을 위한 `practice_transform` 함수 완성하기

A. `flip` 함수 완성하기
- image는 pytorch의 기본 hflip함수를 통해 flip합니다.
- box의 좌표 (x0, y0, x1, y1)을 flip합니다.

B. `resize` 함수 완성하기
- image는 pytorch의 기본 resize함수를 통해 resize합니다.
- box의 좌표가 image의 width, height에 대해 상대적인 위치를 의미하는 0~1의 값을 갖도록 합니다.

In [8]:
import torchvision.transforms.functional as FT


def flip(image, boxes):
    """
    이미지를 수직으로(horizontally) 뒤집는 Augmentation을 적용합니다.
    """
    # 이미지를 수직으로 뒤집습니다.
    new_image = FT.hflip(image)

    # 이미지에 맞추어 box 좌표를 뒤집습니다.
    new_boxes = boxes
    
    # (x0, y0, x1, y1) 중 x0의 좌표를 조정합니다.
    new_boxes[:, 0] = image.width - boxes[:, 0] - 1
    
    # [ToDo]: (x0, y0, x1, y1) 중 x1의 좌표를 조정합니다.
    new_boxes[?, ?] = image.width - ?????????? - ?
    
    # [ToDo]: (new_x1, y0, new_x0 y1)의 형태로 좌표를 조정합니다.
    new_boxes = ?????????????????

    return new_image, new_boxes


def resize(image, boxes, dims=(300, 300)):
    """
    이미지의 크기를 재조정합니다. (300x300)
    """
    # 이미지를 300x300 크기로 바꿉니다.
    new_image = FT.resize(image, dims)

    # 이미지에 맞추어 box 좌표를 조정하기 위해 이미지의 원래 w, h를 저장합니다.
    old_dims = torch.FloatTensor([image.width, image.height, image.width, image.height]).unsqueeze(0)
    
    # [ToDo]: box 좌표를 원래 이미지 크기에 대한 [0,1]의 값으로 조정합니다.
    new_boxes = boxes / ???????
    
    return new_image, new_boxes

In [9]:
import random

def practice_transform(images, boxes):
    
    # [ToDo]: 0.5의 확률로 horizontal flip
    if random.random() < 0.5:
        images, boxes = flip(images, boxes)
    
    # [ToDo]: 300x300 크기로 resize
    images, boxes = resize(images, boxes)
    
    # image를 tensor로 변환
    images = FT.to_tensor(images)
    
    return images, boxes

함수를 완성했다면 Task1의 Class에 추가해 결과를 확인해 보세요.

-------------------
## [Task 3] DataLoader 구성하기

이번에는 DataLoader를 만들어 보도록 하겠습니다. 아래의 코드를 먼저 실행해 봅시다.

In [ ]:
from torch.utils.data.dataloader import DataLoader

train_dataset = PascalVOCDataset(data_folder='./data/VOC', split='TRAIN')
train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)

dataloader가 완성되었으니 첫 번째 batch를 가져와 볼까요?

In [ ]:
#images, boxes, labels, difficulties = next(iter(train_dataloader))

위의 코드에서는 에러가 발생합니다. Dataset은 잘 구성하였으나, 이미지마다 가지고 있는 객체(Object)의 수가 다르기 때문에 이미지마다 서로 다른 dimension의 label 정보를 갖게 되었기 때문입니다.

예를 들어, 2개의 객체를 가지는 이미지의 box 정보는 (2,4)의 차원이지만 4개의 객체를 가진다면 (4,4)가 되고, 두 box를 tensor형태로 쌓아 batch를 구성할 수가 없습니다.

이러한 상황을 해결하기 위해 dataloader에 collate_fn이라는 변수로 배치의 구성에 대해 함수 형태로 지정할 수 있습니다. 간단한 해결 방법으로는 image에 대해서는 문제가 없으니 tensor 형태로 받고, 나머지는 데이터마다 라벨 정보가 따로 저장된 list의 형태로 받는 방법이 있습니다.

In [ ]:
def collate_fn(batch):
    """
    label의 차원이 다른 문제를 해결하기 위해 라벨에 대한 정보는 list로 구성해 batch를 리턴합니다.
    """

    images, boxes, labels, difficulties = [], [], [], []
    
    # [ToDo]: batch 안의 정보들을 순서대로 각각 해당 list에 append합니다.
    for b in batch:
        images.append(???)
        boxes.append(???)
        labels.append(???)
        difficulties.append(???)
    
    # 이미지는 tensor 형태로 바꿉니다. (왜 일까요?)
    images = torch.stack(images, dim=0)

    return images, boxes, labels, difficulties  # tensor (N, 3, 300, 300), 3 lists of N tensors each


`collate_fn`을 완성했다면 dataloader를 구성해 batch를 받아봅시다.

In [ ]:
from torch.utils.data.dataloader import DataLoader

train_dataloader = DataLoader(dataset=train_dataset, batch_size=16, 
                              shuffle=True, collate_fn=collate_fn)

images, boxes, labels, difficulties = next(iter(train_dataloader))

print(boxes)

---------
### <생각해 봅시다>

- TrainSet과 TestSet의 Augmentation은 어떻게 달라야 하나요?
- Random Crop Augmentation을 적용하려면 어떻게 해야 할까요?
------------